# 训练Xception模型，并保存权重

1 使用 Xception

2 冻结训练分类器

3 finetune训练整个网络

### 1 import libs

In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
#config.gpu_options = tf.GPUOptions(allow_growth=True)
set_session(tf.Session(config=config))

### 2 build pretrained model

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input((299, 299, 3))
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(20, activation='softmax')(p_out)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)

parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(optimizer='Adadelta', 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])
parallel_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 20)           20902460    lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
__________

### 3 build param

In [4]:
batch_size = 64
epochs = 1

img_size = (299, 299)

### 4 build data & train

In [5]:
img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory("/home/nvme/CELLS_hls_299/train", 
                                                img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("/home/nvme/CELLS_hls_299/valid",
                                                img_size, shuffle=True, batch_size=batch_size)


parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator))

model.save_weights("Xception_first_train.h5")

Found 110014 images belonging to 20 classes.
Found 13754 images belonging to 20 classes.
Epoch 1/1
1719/1719 [==============================] - 428s 249ms/step - loss: 1.0044 - acc: 0.6967 - val_loss: 1.6638 - val_acc: 0.5240


### 5 Finetune

In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

Using TensorFlow backend.


In [3]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(0.5)(p_out)
predictions = Dense(20, activation='softmax')(p_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception_first_train.h5')

parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(optimizer='Adadelta', 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_2[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

batch_size = 48
epochs = 100
img_size = (299, 299)

img_gen_t = ImageDataGenerator(rotation_range= 90,                            
                               width_shift_range= 0.1,
                               height_shift_range= 0.1,
#                                zoom_range= 0.1,
#                                brightness_range=[0.9, 1.1],
#                               zca_whitening= True,
                               horizontal_flip= True,
                               vertical_flip= True)

train_generator = img_gen_t.flow_from_directory("/home/nvme/CELLS_hls_299/train", 
                                                img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("/home/nvme/CELLS_hls_299/valid", 
                                                img_size, shuffle=True, batch_size=batch_size)

checkpoint = ModelCheckpoint("weights_{epoch:03d}_{val_loss:.4f}.hdf5", monitor='val_loss', verbose=1,
                             save_best_only=False, save_weights_only=True, mode='min')

tensorboard = TensorBoard(log_dir="./logs", histogram_freq=0, batch_size=batch_size, write_graph=True, write_images=True)

callbacks = [checkpoint, tensorboard]

parallel_model.fit_generator(generator=train_generator, steps_per_epoch=len(train_generator), epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=len(valid_generator), callbacks=callbacks)

Found 110014 images belonging to 20 classes.
Found 13754 images belonging to 20 classes.
Epoch 1/100
2292/2292 [==============================] - 1478s 645ms/step - loss: 0.4056 - acc: 0.8607 - val_loss: 0.4205 - val_acc: 0.8560

Epoch 00001: saving model to weights_001_0.4205.hdf5
Epoch 2/100
2292/2292 [==============================] - 1469s 641ms/step - loss: 0.2727 - acc: 0.9028 - val_loss: 0.3680 - val_acc: 0.8790

Epoch 00002: saving model to weights_002_0.3680.hdf5
Epoch 3/100
2292/2292 [==============================] - 1536s 670ms/step - loss: 0.2377 - acc: 0.9150 - val_loss: 0.2468 - val_acc: 0.9115

Epoch 00003: saving model to weights_003_0.2468.hdf5
Epoch 4/100
2292/2292 [==============================] - 1544s 674ms/step - loss: 0.2142 - acc: 0.9230 - val_loss: 0.2194 - val_acc: 0.9215

Epoch 00004: saving model to weights_004_0.2194.hdf5
Epoch 5/100
2292/2292 [==============================] - 1551s 677ms/step - loss: 0.1987 - acc: 0.9279 - val_loss: 0.3138 - val_acc: 0.


Epoch 00042: saving model to weights_042_0.2300.hdf5
Epoch 43/100
2292/2292 [==============================] - 1550s 676ms/step - loss: 0.0711 - acc: 0.9738 - val_loss: 0.2122 - val_acc: 0.9437

Epoch 00043: saving model to weights_043_0.2122.hdf5
Epoch 44/100
1213/2292 [==============>...............] - ETA: 11:23 - loss: 0.0673 - acc: 0.9757

In [ ]:
batch_size = 128
img_size = (299, 299)

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("/home/nvme/CELLS_hls_299/test", 
                                         img_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
test = parallel_model.predict_generator(test_generator, len(test_generator), verbose=1)
test = test[:test_img_nums]
label = test_generator.classes

In [ ]:
print(test.shape)
print(label.shape)

In [ ]:
print(len(test))
print(len(label))

print(test[0])
print(np.argmax(test[0]))

true_nums = 0
total_nums = len(label)
for i in range(len(label)):
    if(label[i] == np.argmax(test[i])):
        true_nums = true_nums + 1

print(true_nums)
print(total_nums)
print(true_nums / total_nums)

### 5 load weights & predict

In [ ]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(1.0)(p_out)
fc_out = Dense(1024, activation='relu')(p_out)
fc_out = Dropout(1.0)(fc_out)
predictions = Dense(3, activation='softmax')(fc_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception.h5')

In [ ]:
img_gen_s = ImageDataGenerator()
submit_generator = img_gen_t.flow_from_directory("../dataset/test", img_size, shuffle=False, batch_size=2)

predictions = []
for i, batch_data in enumerate(submit_generator):
    if (i > 1):
        break
    batch_img_data = np.array(batch_data[0])
    predictions.extend(model.predict_on_batch(batch_img_data))
    print(i)
    
print(predictions)

In [ ]:
print(np.array(predictions))

### 6 write predictions 2 CSV